In [ ]:
import os, sys
from scipy import stats
import numpy as np
import pandas as pd

from   scipy import stats
#-- for ANOVA
import statsmodels.api as sm
from   statsmodels.formula.api import ols
#-- for Tukey
from statsmodels.stats.multicomp import MultiComparison

is_colab = False

if is_colab:
    # create src and upload stat_lib into it
    from src import stat_lib
else:
    sys.path.insert(1, '../src/')
    import stat_lib

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

### Comparando-se o mesmo grupo muitas vezes

  - ANOVA
  - Tukey
  - Dunnett

### ANOVA - Teste de Hipótese de Análise de Variâncias

H0 - hipótese nula:
  - todos grupos têm médias e desvios padrões amostrais próximos ou iguais
  - todas as variáveis randômicas são obtidas por sorteio de uma mesma distribuição
  
Ha - hipótese alternativa:
  - ao menos um grupo tem média e desvio padrão amostral diferentes dos outros
  - ao menos uma variável randômicas foi obtida por sorteio de uma outra distribuição

https://en.wikipedia.org/wiki/One-way_analysis_of_variance

### Novos exemplos
  - dadas 5 hipotéticas amostras
  - com media 140 e variando com delMU
  - com SSD 10, variando com delSSD

In [ ]:
samp_list=[]; mu_list = []; ssd_list = []
N   = 30
n_samp = 5

MU0 = 140; delMUs = [0, -.5, +1, -20, -30]
SSD0 = 10; delSSD = [0, -.2, +.2, -1, +1]

for i in range(n_samp):
    MU = MU0 + delMUs[i]
    SSD = SSD0 + delSSD[i]

    samples = np.random.normal(loc=MU, scale=SSD, size=N)

    samp_list.append(samples)

    mu_list.append(np.mean(samp_list[i]))
    ssd_list.append(np.std(samp_list[i]))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

seqx = np.linspace(70, 180, 100)
colors = ['red', 'blue', 'green', 'brown', 'black']

for i in range(n_samp):

    samples = samp_list[i]
    color = colors[i]

    label = f"{color} {mu_list[i]:.1f} ({ssd_list[i]:.1f})"
    # ax = sns.histplot(samples, stat='density', color=color, alpha=.2, label=label, ax=ax)
    sns.rugplot(samples, color=color, alpha=0.4, label=label, ax=ax)

    plt.vlines(mu_list[i], 0, 0.06, color=color)

    normal_pdf = stats.norm.pdf(seqx, mu_list[i], ssd_list[i])
    sns.lineplot(x=seqx, y=normal_pdf, color=color)

title = 'Distribuições'
plt.legend()
plt.grid()
plt.ylim(0, 0.06)
plt.title(title);


### As distribuições são nomais? teste de Shaprio-Wilkis

In [ ]:
for i in range(n_samp):
    ret, text, text_stat, stat, pvalue = stat_lib.calc_normalidade_SWT(samp_list[i])
    print(text, '\n', text_stat, '\n')

### Alguma distribuição tem média diferente? one-way ANOVA

In [ ]:
ret, text, text_stat, stat, pvalue = stat_lib.test_one_way_ANOVA5(samp_list[0],samp_list[1],samp_list[2],samp_list[3],samp_list[4])
text, text_stat, stat

### Criando um dataframe

In [ ]:
def join_series_by_list(samp_list):

    df_list = []
    for i in range(len(samp_list)):
        samples = samp_list[i]
        label = f"samp{i+1}"

        dic = {'val': samples, 'grupo': label}
        df = pd.DataFrame(dic)

        df_list.append(df)

    df = pd.concat(df_list)
    df.reset_index(inplace=True, drop=True)

    return df


In [ ]:
df = join_series_by_list(samp_list)
print(len(df))
df.tail(3)

## Qual grupo é diferente?

### Tukey test - Post-hoc test

É chamado de test de Tukey, ou método de Tukey, ou teste de significância honesta de Tukey

**POST-HOC** - depois disto

**ANOVA** diz se as distribuições são diferentes mas não diz qual e quanto.

**TUKEY** compara todas as distribuições entre si.

https://en.wikipedia.org/wiki/Tukey%27s_range_test

In [ ]:
plt.figure(figsize=(12,8), dpi=300)

cardata = MultiComparison(df.val, df.grupo)
results = cardata.tukeyhsd()

title  = "Tukey test: multiple comparisons between all pairs"

results.plot_simultaneous()
plt.title(title);

### Combinações

comb(5 2) = 5! / 3! 2! = 5 * 4 / 2 = 20 / 2 = 10

In [ ]:
n_samples = len(samp_list)

count=0
for i in range(n_samples-1):
    for j in range(i+1, n_samples):
        count += 1
        print(f"{count:2}) {i} {j}")

In [ ]:
results.meandiffs

In [ ]:
results.confint

In [ ]:
results.pvalues

In [ ]:
results.summary()

### Dunnett test

  - Um único controle: controle positivo
  - Múltiplos cases
    - controle basal
    - controle negativo
    - 3 drogas
  
Este é o caso quando queremos fazer um experimento de processo anti-inflamatório:
  - Temos uma cultura de células em FBS
  - Adicionamos um sinal inflmatório e medimos uma citocina (IL6, IL1B, TNF-A) após 30 min
  - Após uma hora adicionamos um anti-inflamatório (como dexametasona) e medimos a citocina após 2 horas
  - Repetimos o experimento acima adicionando uma dada droga e medindo o citocina após 2 horas
  
    - Controle: FBS
    - Case: controle positivo - inflamação (perturbação inflamatória)
    - Case: controle negativo - anti-inflamatório (p.ex. dexametazona)
    - Cases: 3 drogas o um veneno em 3 concentrações bem diluídos (ver dose-efeito)
  

In [ ]:
samp_list=[]; MUs = []; SSDs = []
N   = 12
colors = ['yellow', 'red', 'aqua', 'lime', 'forestgreen', 'darkgreen']
sampNames = ['control', 'ctrl-pos', 'ctrl-neg', 'drg1', 'drg2', 'drg3']

# TNF-A ... pmol
MUs  = [20, 400, 40, 420, 240, 90]
SSDs = [10, 25,  10,  25,  20, 11]
n_samples = len(MUs)

for i in range(n_samples):
    samples = np.random.normal(loc=MUs[i], scale=SSDs[i], size=N)
    samp_list.append(samples)


fig, ax = plt.subplots(figsize=(12, 6))
seqx = np.linspace(-20, 480, 100)

for i in range(n_samples):

    label = sampNames[i]
    samples = samp_list[i]
    color = colors[i]

    ax = sns.histplot(samples, stat='density', color=color, alpha=.2, label=label, ax=ax)
    sns.rugplot(samples, color=color, alpha=0.4, ax=ax)

    plt.vlines(MUs[i], 0, 0.03, color=color)

    pdf_normal = stats.norm.pdf(seqx, MUs[i], SSDs[i])
    sns.lineplot(x=seqx, y=pdf_normal, color=color)

title = 'Distribuições'
plt.legend()
plt.ylabel('Densidade de resposta da citocina')
plt.xlabel('Resposta da citocina')
plt.grid()
plt.title(title);


### Barplot

In [ ]:
def join_series_by_list(samp_list, name_list=[]):

    df_list = []
    for i in range(len(samp_list)):
        samples = samp_list[i]

        if name_list == []:
            label = f"samp{i+1}"
        else:
            label = name_list[i]

        dic = {'val': samples, 'grupo': label}
        df = pd.DataFrame(dic)

        df_list.append(df)

    df = pd.concat(df_list)
    df.reset_index(inplace=True, drop=True)

    return df


In [ ]:
sampNames

In [ ]:
df2 = join_series_by_list(samp_list, sampNames)
print(len(df2))
df2.tail(3)

In [ ]:
plt.figure(figsize=(12,6))

ax = sns.barplot(x='grupo', y="val", data=df2, hue='grupo', saturation=0.6, palette=colors, errorbar=('ci', 95))

i=1 # positive control
mu = np.mean(samp_list[i])
plt.hlines(mu, 0, 6, color = colors[i])

plt.ylabel('cytokine (umol)')
plt.xlabel('samples')
plt.title("Teste de 3 drogas x controle pos e neg")


In [ ]:
sns.boxplot(x="grupo", y="val", hue='grupo', data=df2, saturation=0.6, palette=colors);

### 3 gráficos juntos

In [ ]:
ci = 95

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14,6), sharey=True)

ax = axes[0]
ax = sns.barplot(x='grupo', y="val", data=df2, hue='grupo', saturation=0.6, palette=colors, errorbar=('ci', 95), ax=ax)

ax.set_ylabel('cytokine (umol)')
ax.set_xlabel('samples')
ax.set_title("Teste de 3 drogas x controle pos e neg")

ax = axes[1]
sns.boxplot(x="grupo", y="val", hue='grupo', data=df2, saturation=0.6, palette=colors, ax=ax)
ax.set_ylabel('')
ax.set_xlabel('')
ax.set_title("box-plot");

In [ ]:
df2.grupo.unique()

In [ ]:
is_colab

In [ ]:
root_tmp = '/content/tmp' if is_colab else './tmp'

try:
    os.mkdir(root_tmp)
    print(f"Diretório '{root_tmp}' foi criado.")
except:
    print(f"Diretório '{root_tmp}' já existe.")

In [ ]:
fname = 'table.tsv'
filename = os.path.join(root_tmp, fname)

df2.to_csv(filename, sep='\t', index=False)

In [ ]:
df3 = pd.read_csv(filename, sep='\t')
print(df3.shape, N*n_samples)
df3.head(3)

In [ ]:
df3.tail(3)

### Dunnet: todos versos uma referência

In [ ]:
from scipy.stats import dunnett

In [ ]:
df2.grupo.unique()

In [ ]:
control = samp_list[0]
ctr_pos = samp_list[1]
ctr_neg = samp_list[2]
drug1 = samp_list[3]
drug2 = samp_list[4]
drug3 = samp_list[5]


```
def calc_Dunnett_test(val_list:list, control:list):
	res = dunnett(*val_list, control=control)
	return res, res.pvalue
```

In [ ]:
res = stat_lib.dunnett(control, ctr_neg, drug1, drug2, drug3, control=ctr_pos)
pvals = res.pvalue

pvals

In [ ]:
res, pvals = stat_lib.calc_Dunnett_test([control, ctr_neg, drug1, drug2, drug3],  control=ctr_pos)
pvals

In [ ]:
pval_control = pvals[0]
pval_ctr_neg = pvals[1]
pval_drug1 = pvals[2]
pval_drug2 = pvals[3]
pval_drug3 = pvals[4]

In [ ]:
ci = 95

fig, ax = plt.subplots(figsize=(12,6))

ax = sns.barplot(x='grupo', y="val", data=df2, hue='grupo', saturation=0.6, palette=colors, errorbar=('ci', 95), ax=ax)

y1 = 380; dely = 32

#-- barras de erros
for j in range(5):
    pval = pvals[j]

    y1 += dely
    x1 = 0 if j==0 else j+1
    xt = j if j<=1 else 1.5

    ax.hlines(y=y1, xmin=1, xmax=x1, colors='black')

    aster = stat_lib.stat_asteristics(pval)
    text = f'{aster} pval {pval:.1e}'

    ax.text(x=xt, y=y1+10, s=text)

ax.set_ylabel('cytokine (umol)')
ax.set_xlabel('')
ax.set_title(f"Distribuições com {N} amostras\nDunnett-test, comparações contra controle positivo")